<a href="https://colab.research.google.com/github/leyli16/HousingPricePrediction/blob/main/final_project_merged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Packages

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

## Loading data through kaggle setup

In [ ]:
%%capture
!pip instsall kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

# USA Real Estate Data Wrangling and Cleaning

## Loading in USA Real Estate Dataset


In [ ]:
#!/bin/bash
!!kaggle datasets download ahmedshahriarsakib/usa-real-estate-dataset

In [ ]:
!unzip usa-real-estate-dataset.zip

In [ ]:
df_raw = pd.read_csv('realtor-data.zip.csv')
df_raw.head(10)

In [ ]:
df_raw.info()

In [ ]:
df_raw.describe()

In [ ]:
df_raw.describe(include = 'object')

## Cleaning up USA Real Estate data

In [ ]:
df_raw.duplicated().sum()

In [ ]:
# Keep only the necessary columns
df_req_cols = df_raw [['price', 'bed', 'bath', 'acre_lot', 'city', 'state', 'zip_code', 'house_size', 'status']]

In [ ]:
total_missing = df_req_cols.isna().sum()*100/len(df_req_cols)
print('Percentage Missing Value %')
total_missing

In [ ]:
# Drop nulls, reset and drop the index
df_nadropped = df_req_cols.dropna().reset_index(drop=True)
df_nadropped.info()

In [ ]:
total_missing = df_nadropped.isna().sum()*100/len(df_nadropped)
print('Percentage Missing Value %')
total_missing

In [ ]:
# cast date types
df_typed = df_nadropped.astype({'price': 'float', 'bed': 'int', 'bath': 'int', 'acre_lot': 'float', 'city':'str', 'state':'str', 'zip_code':'int', 'house_size': 'float'})

In [ ]:
df_typed.info()

In [ ]:
# Filter rows with status = sold
df_sold = df_typed[df_typed['status'] == 'sold'].drop(columns=['status'])

In [ ]:
# Adding a column price_per_sqft
df_sold['price_per_sqft'] = df_sold['price'] / df_sold['house_size']

In [ ]:
df_sold.head()

In [ ]:
df_real_estate = df_sold

## Loading in Zip code to County Dataset

In [ ]:
#!/bin/bash
!!kaggle datasets download danofer/zipcodes-county-fips-crosswalk

In [ ]:
!unzip zipcodes-county-fips-crosswalk.zip

In [ ]:
zip_county_df = pd.read_csv('ZIP-COUNTY-FIPS_2017-06.csv')
zip_county_df.head(10)

In [ ]:
zip_county_df.info()

## Cleaning up Zip Code to County Dataset

In [ ]:
zip_county_df.duplicated().sum()

In [ ]:
total_missing = zip_county_df.isna().sum() * 100 / len(zip_county_df)
print('Percentage Missing Value %')
total_missing

In [ ]:
# cast date types
zip_county_df = zip_county_df.astype({'ZIP': 'int', 'COUNTYNAME':'str', 'STATE':'str'})

In [ ]:
zip_county_df.info()

In [ ]:
zip_county_df.rename(columns={'ZIP': 'zip_code', 'COUNTYNAME':'county', 'STATE':'state'}, inplace=True)

In [ ]:
zip_county_df.drop(columns=['STCOUNTYFP', 'CLASSFP'], inplace=True)

In [ ]:
# count unique counties per ZIP code
zip_counts = zip_county_df['zip_code'].value_counts()

# Step 1: Find ZIP codes with only one county
zips_one_county = zip_counts[zip_counts == 1].index
# Step 2: Filter out rows with those ZIP codes
zip_county_df = zip_county_df[zip_county_df['zip_code'].isin(zips_one_county)]

In [ ]:
zip_county_df.info()

## Create new column "county" in USA Real Estate Dataset

In [ ]:
# Merge df with zip_county_df to get county info
real_estate_df = pd.merge(df_real_estate, zip_county_df, on='zip_code', how='inner')
real_estate_df.drop(columns=['state_x'], inplace=True)
real_estate_df.rename(columns={'state_y':'state'},inplace=True)
real_estate_df.head(10)

In [ ]:
real_estate_df.shape

In [ ]:
real_estate_df.describe()

In [ ]:
real_estate_df.describe(include = 'object')

## Loading in Zip Code Demographics

In [ ]:
#!/bin/bash
!!kaggle datasets download erdi28/zip-codes-demographics

In [ ]:
!unzip zip-codes-demographics.zip

In [ ]:
zip_code_demographics_df_raw = pd.read_csv('zip_code_demographics.csv')
zip_code_demographics_df_raw.head(10)

In [ ]:
zip_code_demographics_df_raw.info()

In [ ]:
zip_code_demographics_df_raw.describe()

In [ ]:
zip_code_demographics_df_raw.describe(include = 'object')

## Cleaning Zip Code Demographics Data

In [ ]:
zip_code_demographics_df_raw.duplicated().sum()

In [ ]:
# Keep only the necessary columns
zip_code_demographics_df = zip_code_demographics_df_raw [['zip', 'population', 'density', 'dist_highway', 'dist2_large_airport', 'dist_to_shore', 'number_of_business', 'adjusted_gross_income']]

In [ ]:
total_missing = zip_code_demographics_df.isna().sum() * 100 / len(zip_code_demographics_df)
print('Percentage Missing Value %')
total_missing

In [ ]:
# Drop nulls
zip_code_demographics_df = zip_code_demographics_df.dropna()
# Reset and drop the index
zip_code_demographics_df = zip_code_demographics_df.reset_index(drop=True)

In [ ]:
total_missing = zip_code_demographics_df.isna().sum() * 100 / len(zip_code_demographics_df)
print('Percentage Missing Value %')
total_missing

In [ ]:
# cast date types
zip_code_demographics_df = zip_code_demographics_df.astype({'zip': 'int', 'population': 'float', 'density': 'float', 'dist_highway': 'float', 'dist2_large_airport': 'float', 'dist_to_shore': 'float', 'number_of_business': 'float', 'adjusted_gross_income': 'float'})

In [ ]:
zip_code_demographics_df.info()

In [ ]:
zip_code_demographics_df.rename(columns={'zip': 'zip_code'}, inplace=True)

## Merge real_estate_df with zip_code_demographics_df to add the zip code related features

In [ ]:
real_estate_features_df = pd.merge(real_estate_df, zip_code_demographics_df, on='zip_code', how='inner')
real_estate_features_df.head(10)

In [ ]:
real_estate_features_df.info()

In [ ]:
real_estate_features_df.describe()

In [ ]:
real_estate_features_df.describe(include = 'object')

Import Library


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from string import ascii_letters
import matplotlib.pyplot as plt
import datetime as dt
import requests
from lxml import html
import math
import json
import re
import os

# 1. County Demographics Data Cleaning and Wrangling


## 1.1 Loading County Demogrphics Dataset

In [148]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("glozab/county-level-us-demographic-data-1990-2020")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/glozab/county-level-us-demographic-data-1990-2020/versions/1


In [149]:
files = os.listdir(path)
print("Files in dataset:", files)

Files in dataset: ['county_demographics.csv']


In [153]:
file_path = os.path.join(path, 'county_demographics.csv')
county_demographics_df_raw = pd.read_csv(file_path)

print(county_demographics_df_raw.head())

   year  fips  population  w_population  b_population  o_population  \
0  1990  1025       27289         15579         11643            35   
1  1990  1031       40293         32869          6950           160   
2  1990  1041       13598         10068          3516            11   
3  1990  1053       35526         24377         10050          1045   
4  1990  1101      209537        119702         87856           415   

   nh_population  hi_population  na_population  male_population  ...  \
0          27196             93              0            13052  ...   
1          39831            462              0            19673  ...   
2          13576             22              0             6421  ...   
3          35378            148              0            17454  ...   
4         207933           1604              0            98854  ...   

   age9_population_ratio  age10_population_ratio  age11_population_ratio  \
0                0.06263                 0.05552                

## 1.2 County Demographics Dataset Cleaning

In [154]:
county_demographics_df_raw.dtypes

,0
year,int64
fips,int64
population,int64
w_population,int64
b_population,int64
o_population,int64
nh_population,int64
hi_population,int64
na_population,int64
male_population,int64


In [155]:
county_demographics_df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97287 entries, 0 to 97286
Data columns (total 57 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   year                     97287 non-null  int64  
 1   fips                     97287 non-null  int64  
 2   population               97287 non-null  int64  
 3   w_population             97287 non-null  int64  
 4   b_population             97287 non-null  int64  
 5   o_population             97287 non-null  int64  
 6   nh_population            97287 non-null  int64  
 7   hi_population            97287 non-null  int64  
 8   na_population            97287 non-null  int64  
 9   male_population          97287 non-null  int64  
 10  female_population        97287 non-null  int64  
 11  age0_population          97287 non-null  int64  
 12  age1_population          97287 non-null  int64  
 13  age2_population          97287 non-null  int64  
 14  age3_population       

In [156]:
county_demographics_df_raw.isna().sum().sort_values(ascending=False)

,0
year,0
fips,0
population,0
w_population,0
b_population,0
o_population,0
nh_population,0
hi_population,0
na_population,0
male_population,0


### Rename Columns for Consistency

In [157]:
county_demographics_df_raw.columns = county_demographics_df_raw.columns.str.strip().str.lower().str.replace(' ', '_')

In [161]:
# Rename age groups
county_demographics_df_renamed = county_demographics_df_raw
county_demographics_df_renamed.rename(
    columns=
     {
    'w_population': 'caucasion_population',
    'b_population': 'African-American_population',
    'o_population': 'other_race_population',
    'nh_population':'non-Hispanic_population',
    'hi_population':'hispanic_population',
    'na_population':'other_origin_population',
    'w_population_ratio': 'caucasion_population_ratio',
    'b_population_ratio': 'African-American_population_ratio',
    'o_population_ratio': 'other_race_population_ratio',
    'nh_population_ratio':'non-Hispanic_population_ratio',
    'hi_population_ratio':'hispanic_population_ratio',
    'na_population_ratio':'other_origin_population_ratio',
    'age0_population': 'age_less_1_population',
    'age1_population': 'age_1_to_4_population',
    'age2_population': 'age_5_to_9_population',
    'age3_population': 'age_10_to_14_population',
    'age4_population': 'age_15_to_19_population',
    'age5_population': 'age_20_to_24_population',
    'age6_population': 'age_25_to_29_population',
    'age7_population': 'age_30_to_34_population',
    'age8_population': 'age_35_to_39_population',
    'age9_population': 'age_40_to_44_population',
    'age10_population': 'age_45_to_49_population',
    'age11_population': 'age_50_to_54_population',
    'age12_population': 'age_55_to_59_population',
    'age13_population': 'age_60_to_64_population',
    'age14_population': 'age_65_to_69_population',
    'age15_population': 'age_70_to_74_population',
    'age16_population': 'age_75_to_79_population',
    'age17_population': 'age_80_to_84_population',
    'age18_population': 'age_85_and_over_population',
    'age0_population_ratio':'age_less_1_population_ratio',
    'age1_population_ratio':'age_1_to_4_population_ratio',
    'age2_population_ratio':'age_5_to_9_population_ratio',
    'age3_population_ratio':'age_10_to_14_population_ratio',
    'age4_population_ratio':'age_15_to_19_population_ratio',
    'age5_population_ratio':'age_20_to_24_population_ratio',
    'age6_population_ratio':'age_25_to_29_population_ratio',
    'age7_population_ratio':'age_30_to_34_population_ratio',
    'age8_population_ratio':'age_35_to_39_population_ratio',
    'age9_population_ratio':'age_40_to_44_population_ratio',
    'age10_population_ratio':'age_45_to_49_population_ratio',
    'age11_population_ratio':'age_50_to_54_population_ratio',
    'age12_population_ratio':'age_55_to_59_population_ratio',
    'age13_population_ratio':'age_60_to_64_population_ratio',
    'age14_population_ratio':'age_65_to_69_population_ratio',
    'age15_population_ratio':'age_70_to_74_population_ratio',
    'age16_population_ratio':'age_75_to_79_population_ratio',
    'age17_population_ratio':'age_80_to_84_population_ratio',
    'age18_population_ratio':'age_85_and_over_population_ratio'
    }, inplace=True)

In [162]:
county_demographics_df_renamed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97287 entries, 0 to 97286
Data columns (total 57 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   year                               97287 non-null  int64  
 1   fips                               97287 non-null  int64  
 2   population                         97287 non-null  int64  
 3   caucasion_population               97287 non-null  int64  
 4   African-American_population        97287 non-null  int64  
 5   other_race_population              97287 non-null  int64  
 6   non-Hispanic_population            97287 non-null  int64  
 7   hispanic_population                97287 non-null  int64  
 8   other_origin_population            97287 non-null  int64  
 9   male_population                    97287 non-null  int64  
 10  female_population                  97287 non-null  int64  
 11  age_less_1_population              97287 non-null  int

 ### Handle Duplicates

In [164]:
county_demographics_df = county_demographics_df_renamed.drop_duplicates()
county_demographics_df = county_demographics_df.dropna()
county_demographics_df.head()

,year,fips,population,caucasion_population,African-American_population,other_race_population,non-Hispanic_population,hispanic_population,other_origin_population,male_population,...,age_40_to_44_population_ratio,age_45_to_49_population_ratio,age_50_to_54_population_ratio,age_55_to_59_population_ratio,age_60_to_64_population_ratio,age_65_to_69_population_ratio,age_70_to_74_population_ratio,age_75_to_79_population_ratio,age_80_to_84_population_ratio,age_85_and_over_population_ratio
0,1990,1025,27289,15579,11643,35,27196,93,0,13052,...,0.06263,0.05552,0.04998,0.04482,0.04167,0.03767,0.03324,0.02825,0.01843,0.01319
1,1990,1031,40293,32869,6950,160,39831,462,0,19673,...,0.07455,0.06031,0.05180,0.04894,0.04544,0.04145,0.03430,0.02730,0.01660,0.01258
2,1990,1041,13598,10068,3516,11,13576,22,0,6421,...,0.06582,0.05354,0.04574,0.04765,0.05104,0.05273,0.04633,0.03876,0.02456,0.01839
3,1990,1053,35526,24377,10050,1045,35378,148,0,17454,...,0.06820,0.05694,0.05174,0.04771,0.04712,0.04141,0.03502,0.02911,0.01942,0.01368
4,1990,1101,209537,119702,87856,415,207933,1604,0,98854,...,0.06966,0.05219,0.04392,0.04111,0.03967,0.03757,0.02881,0.02294,0.01483,0.01145


In [165]:
county_demographics_df.describe()

,year,fips,population,caucasion_population,African-American_population,other_race_population,non-Hispanic_population,hispanic_population,other_origin_population,male_population,...,age_40_to_44_population_ratio,age_45_to_49_population_ratio,age_50_to_54_population_ratio,age_55_to_59_population_ratio,age_60_to_64_population_ratio,age_65_to_69_population_ratio,age_70_to_74_population_ratio,age_75_to_79_population_ratio,age_80_to_84_population_ratio,age_85_and_over_population_ratio
count,97287.000000,97287.000000,9.728700e+04,9.728700e+04,9.728700e+04,97287.000000,9.728700e+04,9.728700e+04,97287.0,9.728700e+04,...,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000,97287.000000
mean,2005.009436,30412.859950,9.365957e+04,7.529841e+04,1.252406e+04,1129.919044,8.009754e+04,1.356203e+04,0.0,4.598061e+04,...,0.068098,0.067555,0.064839,0.060595,0.054948,0.048050,0.039687,0.030821,0.021798,0.020099
std,8.942825,15147.842995,3.034699e+05,2.259523e+05,5.716518e+04,4815.437070,2.171562e+05,1.058393e+05,0.0,1.489240e+05,...,0.011292,0.010417,0.011904,0.014153,0.014746,0.013994,0.011871,0.009476,0.007503,0.008808
min,1990.000000,1001.000000,5.500000e+01,2.900000e+01,0.000000e+00,0.000000,4.900000e+01,0.000000e+00,0.0,2.900000e+01,...,0.008400,0.013420,0.009150,0.007710,0.005320,0.004840,0.002570,0.000000,0.000000,0.000000
25%,1997.000000,18183.000000,1.094800e+04,9.225500e+03,9.500000e+01,40.000000,1.011600e+04,1.710000e+02,0.0,5.445000e+03,...,0.059970,0.060400,0.055950,0.048960,0.044160,0.038620,0.032010,0.024680,0.016890,0.014210
50%,2005.000000,29179.000000,2.490200e+04,2.166100e+04,8.000000e+02,136.000000,2.327000e+04,6.350000e+02,0.0,1.231900e+04,...,0.068510,0.067780,0.065460,0.060580,0.052770,0.046150,0.038470,0.029950,0.021000,0.018590
75%,2013.000000,45083.000000,6.347900e+04,5.624700e+04,5.440000e+03,550.000000,5.989000e+04,3.148000e+03,0.0,3.135050e+04,...,0.075770,0.074500,0.073120,0.071100,0.064605,0.055630,0.045940,0.035970,0.025670,0.024350
max,2020.000000,99999.000000,1.009486e+07,7.301940e+06,1.437609e+06,172454.000000,5.508975e+06,4.868179e+06,0.0,4.975410e+06,...,0.160000,0.155960,0.153010,0.258060,0.181820,0.183910,0.183580,0.143400,0.144440,0.195400


## 1.2 Loading Fips Code DataSet




In [166]:
# Loading county df data from google drive
fips_df = pd.read_csv('/content/drive/MyDrive/CIS 5450 Group Project/data_sets/state_and_county_fips_master.csv')

In [167]:
fips_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3195 entries, 0 to 3194
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   fips    3195 non-null   int64 
 1   name    3195 non-null   object
 2   state   3143 non-null   object
dtypes: int64(1), object(2)
memory usage: 75.0+ KB


## 1.3 Merging the county_demographics_df with fips_df

In [168]:
county_demographics_merged_df = county_demographics_df.merge(
    fips_df,
    on='fips',
    how='left'
)
county_demographics_merged_df.head()

,year,fips,population,caucasion_population,African-American_population,other_race_population,non-Hispanic_population,hispanic_population,other_origin_population,male_population,...,age_50_to_54_population_ratio,age_55_to_59_population_ratio,age_60_to_64_population_ratio,age_65_to_69_population_ratio,age_70_to_74_population_ratio,age_75_to_79_population_ratio,age_80_to_84_population_ratio,age_85_and_over_population_ratio,name,state
0,1990,1025,27289,15579,11643,35,27196,93,0,13052,...,0.04998,0.04482,0.04167,0.03767,0.03324,0.02825,0.01843,0.01319,Clarke County,AL
1,1990,1031,40293,32869,6950,160,39831,462,0,19673,...,0.05180,0.04894,0.04544,0.04145,0.03430,0.02730,0.01660,0.01258,Coffee County,AL
2,1990,1041,13598,10068,3516,11,13576,22,0,6421,...,0.04574,0.04765,0.05104,0.05273,0.04633,0.03876,0.02456,0.01839,Crenshaw County,AL
3,1990,1053,35526,24377,10050,1045,35378,148,0,17454,...,0.05174,0.04771,0.04712,0.04141,0.03502,0.02911,0.01942,0.01368,Escambia County,AL
4,1990,1101,209537,119702,87856,415,207933,1604,0,98854,...,0.04392,0.04111,0.03967,0.03757,0.02881,0.02294,0.01483,0.01145,Montgomery County,AL


### Sanity Check After Merging

In [169]:
print(county_demographics_merged_df.columns)         # See all columns
print(county_demographics_merged_df[['fips', 'name']].head())  # Confirm correct matches
print(county_demographics_merged_df.isna().sum())    # Check for unmatched FIPS

Index(['year', 'fips', 'population', 'caucasion_population',
       'African-American_population', 'other_race_population',
       'non-Hispanic_population', 'hispanic_population',
       'other_origin_population', 'male_population', 'female_population',
       'age_less_1_population', 'age_1_to_4_population',
       'age_5_to_9_population', 'age_10_to_14_population',
       'age_15_to_19_population', 'age_20_to_24_population',
       'age_25_to_29_population', 'age_30_to_34_population',
       'age_35_to_39_population', 'age_40_to_44_population',
       'age_45_to_49_population', 'age_50_to_54_population',
       'age_55_to_59_population', 'age_60_to_64_population',
       'age_65_to_69_population', 'age_70_to_74_population',
       'age_75_to_79_population', 'age_80_to_84_population',
       'age_85_and_over_population', 'caucasion_population_ratio',
       'African-American_population_ratio', 'other_race_population_ratio',
       'non-Hispanic_population_ratio', 'hispanic_population

## 1.4 Cleaning the Merged Data

Checking for unmatched FIPS — that output tells us 174 rows in the county_demographics_df didn't find a match in fips_df, because name, state, and any other info from fips_df came back as NaN.

In [170]:
unmatched = county_demographics_merged_df[county_demographics_merged_df['name'].isna()]
print(unmatched['fips'].unique()[:10])

[ 2910  4910 51917  8911  2201  8912  8913  2280  2232  2010]


In [171]:
county_demographics_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97287 entries, 0 to 97286
Data columns (total 59 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   year                               97287 non-null  int64  
 1   fips                               97287 non-null  int64  
 2   population                         97287 non-null  int64  
 3   caucasion_population               97287 non-null  int64  
 4   African-American_population        97287 non-null  int64  
 5   other_race_population              97287 non-null  int64  
 6   non-Hispanic_population            97287 non-null  int64  
 7   hispanic_population                97287 non-null  int64  
 8   other_origin_population            97287 non-null  int64  
 9   male_population                    97287 non-null  int64  
 10  female_population                  97287 non-null  int64  
 11  age_less_1_population              97287 non-null  int

Drop the unmatched rows.

In [172]:
county_demographics_merged_df = county_demographics_merged_df[county_demographics_merged_df['name'].notna()].reset_index(drop=True)
county_demographics_merged_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97113 entries, 0 to 97112
Data columns (total 59 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   year                               97113 non-null  int64  
 1   fips                               97113 non-null  int64  
 2   population                         97113 non-null  int64  
 3   caucasion_population               97113 non-null  int64  
 4   African-American_population        97113 non-null  int64  
 5   other_race_population              97113 non-null  int64  
 6   non-Hispanic_population            97113 non-null  int64  
 7   hispanic_population                97113 non-null  int64  
 8   other_origin_population            97113 non-null  int64  
 9   male_population                    97113 non-null  int64  
 10  female_population                  97113 non-null  int64  
 11  age_less_1_population              97113 non-null  int

In [173]:
county_demographics_merged_df = county_demographics_merged_df.rename(columns={'name': 'county'})
county_demographics_merged_df = county_demographics_merged_df.dropna()
county_demographics_merged_df = county_demographics_merged_df.drop_duplicates()
county_demographics_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97113 entries, 0 to 97112
Data columns (total 59 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   year                               97113 non-null  int64  
 1   fips                               97113 non-null  int64  
 2   population                         97113 non-null  int64  
 3   caucasion_population               97113 non-null  int64  
 4   African-American_population        97113 non-null  int64  
 5   other_race_population              97113 non-null  int64  
 6   non-Hispanic_population            97113 non-null  int64  
 7   hispanic_population                97113 non-null  int64  
 8   other_origin_population            97113 non-null  int64  
 9   male_population                    97113 non-null  int64  
 10  female_population                  97113 non-null  int64  
 11  age_less_1_population              97113 non-null  int

In [174]:
county_demographics_merged_df.head()

,year,fips,population,caucasion_population,African-American_population,other_race_population,non-Hispanic_population,hispanic_population,other_origin_population,male_population,...,age_50_to_54_population_ratio,age_55_to_59_population_ratio,age_60_to_64_population_ratio,age_65_to_69_population_ratio,age_70_to_74_population_ratio,age_75_to_79_population_ratio,age_80_to_84_population_ratio,age_85_and_over_population_ratio,county,state
0,1990,1025,27289,15579,11643,35,27196,93,0,13052,...,0.04998,0.04482,0.04167,0.03767,0.03324,0.02825,0.01843,0.01319,Clarke County,AL
1,1990,1031,40293,32869,6950,160,39831,462,0,19673,...,0.05180,0.04894,0.04544,0.04145,0.03430,0.02730,0.01660,0.01258,Coffee County,AL
2,1990,1041,13598,10068,3516,11,13576,22,0,6421,...,0.04574,0.04765,0.05104,0.05273,0.04633,0.03876,0.02456,0.01839,Crenshaw County,AL
3,1990,1053,35526,24377,10050,1045,35378,148,0,17454,...,0.05174,0.04771,0.04712,0.04141,0.03502,0.02911,0.01942,0.01368,Escambia County,AL
4,1990,1101,209537,119702,87856,415,207933,1604,0,98854,...,0.04392,0.04111,0.03967,0.03757,0.02881,0.02294,0.01483,0.01145,Montgomery County,AL


In [175]:
# filter demographics data for year of 2020
county_demographics_merged_df = county_demographics_merged_df[county_demographics_merged_df['year'] == 2020].reset_index(drop=True)
county_demographics_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3141 entries, 0 to 3140
Data columns (total 59 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   year                               3141 non-null   int64  
 1   fips                               3141 non-null   int64  
 2   population                         3141 non-null   int64  
 3   caucasion_population               3141 non-null   int64  
 4   African-American_population        3141 non-null   int64  
 5   other_race_population              3141 non-null   int64  
 6   non-Hispanic_population            3141 non-null   int64  
 7   hispanic_population                3141 non-null   int64  
 8   other_origin_population            3141 non-null   int64  
 9   male_population                    3141 non-null   int64  
 10  female_population                  3141 non-null   int64  
 11  age_less_1_population              3141 non-null   int64

## 1.5 Merge with real_estate_features_df to add the new features

In [176]:
# Merge with real_estate_features_df
real_estate_features_final_df = pd.merge(real_estate_features_df, county_demographics_merged_df, on=['county','state'], how='inner')
real_estate_features_final_df.head()

,price,bed,bath,acre_lot,city,zip_code,house_size,price_per_sqft,county,state,...,age_40_to_44_population_ratio,age_45_to_49_population_ratio,age_50_to_54_population_ratio,age_55_to_59_population_ratio,age_60_to_64_population_ratio,age_65_to_69_population_ratio,age_70_to_74_population_ratio,age_75_to_79_population_ratio,age_80_to_84_population_ratio,age_85_and_over_population_ratio
0,215000.0,3,2,0.19,Chicopee,1020,1828.0,117.614880,Hampden County,MA,...,0.05645,0.05767,0.06395,0.06911,0.06719,0.05734,0.04590,0.03111,0.01983,0.02441
1,389900.0,3,1,24.00,Northampton,1062,1559.0,250.096216,Hampshire County,MA,...,0.04859,0.04885,0.05638,0.06447,0.06512,0.06218,0.05229,0.03073,0.01899,0.02215
2,269900.0,2,1,0.46,South Hadley,1075,1312.0,205.716463,Hampshire County,MA,...,0.04859,0.04885,0.05638,0.06447,0.06512,0.06218,0.05229,0.03073,0.01899,0.02215
3,314900.0,5,2,0.28,Chicopee,1013,2219.0,141.910771,Hampden County,MA,...,0.05645,0.05767,0.06395,0.06911,0.06719,0.05734,0.04590,0.03111,0.01983,0.02441
4,244999.0,4,1,0.23,Springfield,1104,1285.0,190.660700,Hampden County,MA,...,0.05645,0.05767,0.06395,0.06911,0.06719,0.05734,0.04590,0.03111,0.01983,0.02441


In [177]:
real_estate_features_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451596 entries, 0 to 451595
Data columns (total 74 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   price                              451596 non-null  float64
 1   bed                                451596 non-null  int64  
 2   bath                               451596 non-null  int64  
 3   acre_lot                           451596 non-null  float64
 4   city                               451596 non-null  object 
 5   zip_code                           451596 non-null  int64  
 6   house_size                         451596 non-null  float64
 7   price_per_sqft                     451596 non-null  float64
 8   county                             451596 non-null  object 
 9   state                              451596 non-null  object 
 10  population_x                       451596 non-null  float64
 11  density                            4515

# 2. FEMA Disaster Declarations Data Cleaning and Wrangling

In this section, we clean and wrangle FEMA disaster declaration data to prepare it for analysis. The raw data is filtered to include only disasters from the past 10 years, joined with ZIP codes using a county-level crosswalk, and aggregated to generate a ZIP-level disaster count feature for downstream housing price modeling.

The final output table, `zip_disaster_counts`, contains the number of FEMA disaster declarations per ZIP code over the last decade. This will be used as a static risk feature in our housing price model.





##2.1 Import Required Libraries



In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests

## 2.2 Load FEMA Disaster Declarations Data

Note: We loaded the full dataset here to understand the available columns. In the next step, we’ll select only the relevant ones.

In [ ]:
fema_url = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries.csv"
fema_file_path = "DisasterDeclarationsSummaries.csv"

# Download the file if it doesn't already exist
if not os.path.exists(fema_file_path):
    response = requests.get(fema_url)
    with open(fema_file_path, "wb") as file:
        file.write(response.content)

# Load the entire dataset first
fema_raw_df = pd.read_csv(fema_file_path)

# Preview full column names
fema_raw_df.head()


## 2.3 Select Relevant Columns：

In [ ]:
# Select only the columns we need
fema_df = fema_raw_df[[
    'disasterNumber', 'state', 'designatedArea', 'declarationDate',
    'fipsStateCode', 'fipsCountyCode'
]].copy()

# Convert to datetime
fema_df['declarationDate'] = pd.to_datetime(fema_df['declarationDate'])

# Preview cleaned dataframe
fema_df.head()


## 2.4 Filter Data from the Last 10 Years

In [ ]:
from pandas.tseries.offsets import DateOffset

# Today's date and dynamic 10-year cutoff
today = pd.Timestamp.now(tz='UTC')
cutoff_date = today - DateOffset(years=10)

# Filter FEMA data
fema_recent_df = fema_df[fema_df['declarationDate'] >= cutoff_date].copy()

# Check the date range
print("Earliest disaster:", fema_recent_df['declarationDate'].min())
print("Latest disaster:", fema_recent_df['declarationDate'].max())
print("Total records:", fema_recent_df.shape[0])

# Preview
fema_recent_df.head()

## 2.5 Clean FEMA Data (Standardize & Prepare for Join)

In [ ]:
# Construct full 5-digit county-level FIPS code
fema_recent_df['fips'] = (
    fema_recent_df['fipsStateCode'].astype(str).str.zfill(2) +
    fema_recent_df['fipsCountyCode'].astype(str).str.zfill(3)
)

# Preview
fema_recent_df.head()

## 2.6 Load County-to-ZIP Mapping Table from Kaggle

In [ ]:
# Install kagglehub if not already installed
!pip install kagglehub --quiet

import kagglehub
import os
import pandas as pd

# Download dataset from Kaggle
path = kagglehub.dataset_download("danofer/zipcodes-county-fips-crosswalk")

print("Path to dataset files:", path)

# List files
files = os.listdir(path)
print("Files in dataset:", files)

# Load the crosswalk CSV
file_path = os.path.join(path, "ZIP-COUNTY-FIPS_2017-06.csv")
zip_crosswalk_df = pd.read_csv(file_path)

zip_crosswalk_df['fips'] = zip_crosswalk_df['STCOUNTYFP'].astype(str).str.zfill(5)


# Preview
zip_crosswalk_df.head()



## 2.7 Merge FEMA with ZIP (via FIPS)

In [ ]:
fema_with_zip = fema_recent_df.merge(
    zip_crosswalk_df[['ZIP', 'fips']],
    on='fips',
    how='left'
)

# Drop FEMA rows with no ZIP match
#fema_with_zip = fema_with_zip.dropna(subset=['ZIP'])

# Preveiw
fema_with_zip.head()

In [ ]:
print(f"Total FEMA records after 10-year filter: {fema_recent_df.shape[0]}")
print(f"FEMA records after ZIP merge: {fema_with_zip.shape[0]}")
print(f"Number of FEMA records with ZIP: {fema_with_zip['ZIP'].notna().sum()}")
print(f"Number of FEMA records missing ZIP: {fema_with_zip['ZIP'].isna().sum()}")


## 2.8: Merge Validation — FEMA and ZIP Mapping

In this section, we validate whether FEMA disaster records were successfully joined with ZIP codes using fips codes. We check the match rate, ZIP assignment distribution, and inspect records with missing ZIPs.



### 2.8.1 Match Rate Check
This checks whether the merge was successful for a significant portion of the data.

In [ ]:
total_records = fema_with_zip.shape[0]
matched_zip_count = fema_with_zip['ZIP'].notna().sum()
missing_zip_count = fema_with_zip['ZIP'].isna().sum()
match_rate = matched_zip_count / total_records

print(f"Total FEMA records (10 yrs):     {total_records}")
print(f"Matched ZIPs:                    {matched_zip_count}")
print(f"Missing ZIPs:                    {missing_zip_count}")
print(f"ZIP Match Rate:                  {match_rate:.2%}")


### 2.8.2 ZIP Duplication per FIPS

One county (FIPS) may correspond to multiple ZIP codes. We inspect the distribution to ensure the ZIP mapping is reasonable.


In [ ]:
zip_per_fips = (
    fema_with_zip[['fips', 'ZIP']]
    .drop_duplicates()
    .groupby('fips')
    .size()
    .reset_index(name='zip_count')
)

zip_per_fips['zip_count'].describe()


###2.8.3 Missing ZIPs (Rows That Didn't Match)
Identify FEMA records from counties not covered in the ZIP mapping dataset.

In [ ]:
missing_zip_df = fema_with_zip[fema_with_zip['ZIP'].isna()]
print(f"Number of FEMA records without ZIP after merge: {missing_zip_df.shape[0]}")
missing_zip_df[['state', 'designatedArea', 'fips']].drop_duplicates().head(10)


#### 2.8.3 ZIP Missing Analysis

After merging FEMA disaster data with ZIP codes using FIPS codes, **934** records could not be matched.

Top unmatched designatedArea examples:
*   Resighini Rancheria (Indian Reservation), CA
*   Nez Perce Indian Reservation, ID
*   Yakama Reservation, WA
*   Warm Springs Indian Reservation, OR
*   Crow Indian Reservation, MT
*   San Carlos Indian Reservation, AZ
*   Hopland Rancheria (Indian Reservation), CA



**Possible reasons:**
- Indian Reservations and tribal lands often don't align with standard county or ZIP boundaries

- Non-standard area names that don’t match ZIP datasets (e.g., Rancheria, Reservation)

**Action:** These records can be excluded


In [ ]:
# Drop FEMA records with missing ZIPs
fema_with_zip['zip_matched'] = fema_with_zip['ZIP'].notna()
fema_matched_df = fema_with_zip[fema_with_zip['zip_matched']].copy().reset_index(drop=True)

# Preview
print(fema_matched_df.shape)
fema_matched_df.head()

###2.8.4 FEMA-ZIP Merge Summary

In [ ]:
# Total rows
rows_before = fema_with_zip.shape[0]
rows_after = fema_matched_df.shape[0]

# ZIP matching stats
matched_rows = fema_with_zip['ZIP'].notna().sum()
missing_rows = fema_with_zip['ZIP'].isna().sum()
match_rate = matched_rows / rows_before
final_match_rate = 1.0  # because all rows in fema_matched_df have ZIPs

# ZIPs per FIPS (before filtering)
zip_per_fips = (
    fema_with_zip[['fips', 'ZIP']]
    .drop_duplicates()
    .groupby('fips')
    .size()
    .reset_index(name='zip_count')
)

# Summary stats
print("FEMA-ZIP Merge Summary:")
print(f"Total FEMA records (before filtering):   {rows_before}")
print(f"Total FEMA records (after filtering):    {rows_after}")
print(f"Matched ZIPs:                             {matched_rows}")
print(f"Missing ZIPs:                             {missing_rows}")
print(f"ZIP Match Rate:                           {match_rate:.2%}")
print(f"ZIP Match Rate (final dataset):           {final_match_rate:.2%}")
print(f"Unique county FIPS:                       {zip_per_fips.shape[0]}")
print(f"Median ZIPs per county:                   {zip_per_fips['zip_count'].median():.0f}")
print(f"Max ZIPs per county:                      {zip_per_fips['zip_count'].max()}")


##2.10 Disaster Count Aggregation by ZIP (Last 10 Years)

To support downstream analysis, we compute the number of FEMA disaster declarations in the past 10 years for each ZIP code. This gives a static risk exposure feature per ZIP that can be merged with housing data.

The aggregation is based on the cleaned FEMA dataset `fema_matched_df`, which includes only matched ZIP-level records in past 10 years.

In [ ]:
# Count the number of disasters per ZIP code
zip_disaster_counts = (
    fema_matched_df.groupby('ZIP')['disasterNumber']
    .nunique()
    .reset_index(name='disaster_count_10yrs')
)

# Format ZIPs as 5-digit strings
# zip_disaster_counts['ZIP'] = zip_disaster_counts['ZIP'].astype(int).astype(str).str.zfill(5)

# Make sure ZIP is stored as integer (drop decimals)
zip_disaster_counts['ZIP'] = zip_disaster_counts['ZIP'].astype(int)

# Reset index for cleanliness
zip_disaster_counts = zip_disaster_counts.reset_index(drop=True)

# Preview top ZIPs by disaster count
zip_disaster_counts.sort_values(by='disaster_count_10yrs', ascending=False).head(10)


## Summary of Disaster Count Aggregation

The resulting table `zip_disaster_counts` contains the number of unique FEMA disaster declarations per ZIP code over the past 10 years. Each row represents a ZIP and its corresponding disaster exposure:

- **ZIP**: 5-digit ZIP code (int)
- **disaster_count_10yrs**: Total number of FEMA disaster declarations in that ZIP over the past 10 years

This table will serve as a **static risk feature** that can be joined with the main housing dataset using the ZIP code as key.


# HPI Dataframe by Zip

## Uploading Data

In [ ]:
file_path = '/content/drive/MyDrive/hpi_at_bdl_zip5.csv'
# Read the file
hpi_df = pd.read_csv(file_path)
hpi_df.info()

In [ ]:
# Drop meaningless columns
columns_to_drop = ["HPI with 2000 base", "Unnamed: 6","Unnamed: 7","Unnamed: 8","Unnamed: 9","Unnamed: 10","Unnamed: 11","Unnamed: 12",]
hpi_df = hpi_df.drop(columns=columns_to_drop)

# Convert "Five-Digit ZIP Code" to string and rename the column to zip_code
hpi_df["Five-Digit ZIP Code"] = hpi_df["Five-Digit ZIP Code"].apply(lambda x: f"{int(x):05}")
hpi_df = hpi_df.rename(columns={"Five-Digit ZIP Code": "zip_code"})

# Filter timeline from 1990-2020
filtered_hpi_df = hpi_df[(hpi_df['Year'] >= 1990) & (hpi_df['Year'] <= 2020)]

# Rename columns
filtered_hpi_df = filtered_hpi_df.rename(columns={"Annual Change (%)": "annual_change"})
filtered_hpi_df = filtered_hpi_df.rename(columns={"HPI with 1990 base": "hpi_1990"})


filtered_hpi_df = filtered_hpi_df.reset_index(drop=True)
filtered_hpi_df.info()

In [ ]:
filtered_hpi_df.head(10)

## Check Data Quality for Housing Price Time Series

In [ ]:
# Check the number of rows in dataframe
print("Rows in filtered_hpi_df before dropping NA:\n", len(filtered_hpi_df))
print("\n")

# Check Nulls or missing values
missing_values = filtered_hpi_df.isna().sum()
missing_percentage = (missing_values / len(filtered_hpi_df)) * 100

print("Missing value breakdown:\n", missing_values)
print("\n")
print("Missing Percentage:\n", missing_percentage)
print("\n")
# Find rows where only 'HPI' is NaN and other columns have non-NaN values
only_hpi_na = filtered_hpi_df[filtered_hpi_df['HPI'].isna() & filtered_hpi_df.drop(columns=['HPI']).notna().all(axis=1)]
# Display how many rows have only 'HPI' NaN and not NaN in other columns
print(f"Rows with NaN in 'HPI' but not in other columns: {only_hpi_na.shape[0]}")

In [ ]:
# Drop hpi_1990 column since there is a large amount of missing value and won't be good reference data point
filtered_hpi_df = filtered_hpi_df.drop(columns=['hpi_1990'])
filtered_hpi_df.info()

In [ ]:
# Drop NAs in HPI and using HPI as major housing price prediction
# // Potetial Alternative Treatment on Missing values is to get Average of HPI for ZIPs if they are in the same MSA or County

cleaned_hpi_df = filtered_hpi_df.dropna(subset=['annual_change', 'HPI'], how='all')
print("Rows in cleaned_hpi_df after dropping NA:\n", len(cleaned_hpi_df))

cleaned_hpi_df.head()

## Transform Data for Zip code level with annual HPI info

In [ ]:
# Convert the HPI table by Zip level with annual HPI info appended for each year


# Pivot the data: reshape so each year’s values become their own set of columns
pivoted_hpi_df = filtered_hpi_df.pivot_table(
    index='zip_code',
    columns='Year',
    values=['HPI', 'annual_change']
)

# Flatten the multi-level column index
pivoted_hpi_df.columns = [f'{metric}_{year}' for metric, year in pivoted_hpi_df.columns]

# Reset index to turn zip_code back into a column
pivoted_hpi_df = pivoted_hpi_df.reset_index()

# Display the result
print(pivoted_hpi_df.head())
print("\n")
print("Rows in pivoted_hpi_df", len(pivoted_hpi_df))

In [ ]:
# merged_hpi_df = pd.merge(merged_zip_county_MSA_df, pivoted_hpi_df, left_on='zip_code', right_on='zip_code', how='inner')

merged_hpi_df = pd.merge(
    merged_zip_county_MSA_df,
    pivoted_hpi_df,
    left_on='zip_code',
    right_on='zip_code',
    how='outer',  # use 'outer' to catch all unmatched rows from both sides
    indicator=True
)

In [ ]:
# Rows only from left DataFrame (no match from pivoted_hpi_df)
unmatched_left = merged_hpi_df[merged_hpi_df['_merge'] == 'left_only']

# Rows only from right DataFrame (no match from merged_zip_county_MSA_df)
unmatched_right = merged_hpi_df[merged_hpi_df['_merge'] == 'right_only']

# Zip codes from left DataFrame that had no match in right
print("Unmatched zip codes from left DataFrame:")
print(unmatched_left['zip_code'].unique())

# Zip codes from right DataFrame that had no match in left
print("Unmatched zip codes from right DataFrame:")
print(unmatched_right['zip_code'].unique())
print("\n")

missing_from_right = (merged_hpi_df['_merge'] == 'left_only').sum()
missing_from_left = (merged_hpi_df['_merge'] == 'right_only').sum()
print(f"Zip codes missing from pivoted_hpi_df (right): {missing_from_right}")
print(f"Zip codes missing from merged_zip_county_MSA_df (left): {missing_from_left}")

# merged_hpi_df.drop(columns=['_merge'], inplace=True)

In [ ]:
print(merged_hpi_df.head())
print("\n")
print("Rows in pivoted_hpi_df", len(pivoted_hpi_df))

## Merging HPI time series with Demographic and FEMA dataframe

In [ ]:
# Merge HPI with Demographic and FEMA dataframe

